In [123]:
import pandas as pd
import numpy as np
import networkx as nx
import json
import requests
import os

import tweepy
from tweepy import Client


api = tweepy.Client(bearer_token=token, consumer_key=key, consumer_secret=secret, 
              return_type=requests.Response,
              wait_on_rate_limit=True)

In [131]:
already_processed = []

In [146]:
label = 'human'



In [147]:
for name in os.listdir(label):
    if os.path.isdir(name):
        continue
    this_acc = name.replace('likes_human_','')
    already_processed.append(int(this_acc))

    path =  label + '/' + name
    edgelist = pd.read_pickle(path)
    
    if edgelist.shape[0] == 0:
        continue
    
    to_merge = edgelist.loc[(edgelist.originating_follower_count.notna()) 
                        | (edgelist.originating_following_count.notna())].drop(columns='receiving_id')

    agg = edgelist.groupby(['originating_id', 'receiving_id']).size().reset_index().rename(columns={
        0: 'edge_weight'
    })

    agg = agg.sort_values(by='edge_weight', ascending=False)

    merged = agg.merge(to_merge, on='originating_id', how='left').drop_duplicates()
    merged['receiving_following_count'] = np.nan
    merged['receiving_follower_count'] = np.nan

    returned = add_follow_counts(merged)

    returned.to_pickle('final_data/' + label + '/' + this_acc)

In [141]:
def add_follow_counts(edgelist):
    edgelist = edgelist[['originating_id', 'receiving_id', 'edge_weight']]
    all_temp = pd.DataFrame(columns = ['originating_id', 'originating_follower_count',
       'originating_following_count'])
    all_temp2 = pd.DataFrame(columns = ['receiving_id', 'receiving_follower_count',
       'receiving_following_count'])

    # Add follower/following counts
    unique_ids = edgelist['originating_id'].append(edgelist['receiving_id']).unique()

    iterations = int(len(unique_ids)/100) + 1
    for i in range(iterations):
        left = i*100
        right = (i+1)*100

        temps = unique_ids[left:right]
        if len(temps) == 0:
            continue

        r = api.get_users(ids=list(temps), user_fields='public_metrics')

        followers = []
        following = []
        for acc in r.json()['data']:
            followers.append(acc['public_metrics']['followers_count'])
            following.append(acc['public_metrics']['following_count'])

        temp = pd.DataFrame([list(temps), followers, following]).transpose().rename(columns={
            0: 'originating_id',
            1: 'originating_follower_count',
            2:  'originating_following_count'
        })
        temp.originating_id = temp.originating_id.astype(str)

        temp2 = pd.DataFrame([list(temps), followers, following]).transpose().rename(columns={
            0: 'receiving_id',
            1: 'receiving_follower_count',
            2:  'receiving_following_count'
        })
        temp2.receiving_id = temp2.receiving_id.astype(str)


        all_temp = pd.concat([all_temp, temp])
        all_temp2 = pd.concat([all_temp2, temp2])

        with_counts = edgelist.merge(all_temp, on='originating_id', how='left')
        with_counts = with_counts.merge(all_temp2, on='receiving_id', how='left')
        
        with_counts = with_counts.drop_duplicates()
    return with_counts